# <center>Análise de cancelamento de vendas</center>
___

### 1.0 Bibliotecas

In [ ]:
#transformação
import pandas as pd
import numpy as np
from datetime import datetime
import warnings 
warnings.filterwarnings('ignore')

# visualização
import matplotlib.pyplot as plt
import seaborn as sns

#classificação
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, KFold
from catboost import CatBoostClassifier

#serializar modelo
import pickle

### 2.0 Importar dados

In [ ]:
#importar a base
base_varejo = pd.read_csv('online_retail_II.csv', sep = ';', decimal=',', header=0, encoding='latin1')

In [ ]:
#visualizar base
base_varejo

In [ ]:
#shape
base_varejo.shape

In [ ]:
#verificar tipo de dados
base_varejo.info()

In [ ]:
#resumo quantitativo
base_varejo.describe()

In [ ]:
#verificar obs vazias
base_varejo.isna().sum()

#Descrição           4382
#ClienteID         243007

In [ ]:
#excluir as linhas com ClienteID vazias
base_varejo.dropna(subset=['ClienteID'], inplace=True)
base_varejo.reset_index(drop=True, inplace=True)

In [ ]:
#Retirando da base código de estoque 'M', 'AMAZONFEE'
base_varejo = base_varejo[~base_varejo['Código_Estoque'].isin(['M', 'AMAZONFEE'])]
base_varejo.reset_index(drop=True, inplace=True)

In [ ]:
#alterar tipo de variável
base_varejo['Data_Fatura'] = pd.to_datetime(base_varejo['Data_Fatura'], infer_datetime_format = True, format="%d/%m/%Y")
base_varejo['ClienteID'] = round(base_varejo['ClienteID'])
base_varejo['ClienteID'] = base_varejo['ClienteID'].astype('int') #arredondar valor
base_varejo['ClienteID'] = base_varejo['ClienteID'].astype('str')

In [ ]:
#criar coluna de mês e ano
base_varejo['Mês'] = base_varejo['Data_Fatura'].dt.month
base_varejo['Ano'] = base_varejo['Data_Fatura'].dt.year

In [ ]:
# Função para converter uma string em um objeto datetime
def converter_para_hora(hora_str):
    return datetime.strptime(hora_str, '%H:%M:%S')

# Aplicar a função à coluna 'Hora_Fatura' e criar uma nova coluna 'hora'
base_varejo['Hora_Fatura'] = base_varejo['Hora_Fatura'].apply(converter_para_hora)

In [ ]:
#verificar obs vazias
base_varejo.isna().sum()

In [ ]:
#verificar tipo de dados
base_varejo.info()

In [ ]:
#visualizar base
base_varejo

### 3.0 EDA - Análise exploratória de dados

In [ ]:
#identificando as transações canceladas
base_varejo['Canceladas'] = ['Sim' if  base_varejo['Fatura'][i][0] == 'C' else 'Não' for i in range(0, len(base_varejo))]

In [ ]:
#Vendas
base_varejo['Vendas'] = base_varejo['Quantidade']*base_varejo['Preço']

In [ ]:
#métricas ipc
print('Total de transações canceladas: ' + str(base_varejo['Canceladas'].value_counts().get('Sim', 0)))
print('\nProporção de transações canceladas: ' + str((base_varejo['Canceladas'].value_counts().get('Sim', 0) / len(base_varejo['Canceladas'])) * 100))
print('\nValor de transações canceladas: ' + str(base_varejo['Vendas'][base_varejo['Canceladas'] == 'Sim'].sum()))
print('\nProporção de transações canceladas: ' + str((base_varejo['Vendas'][base_varejo['Canceladas'] == 'Sim'].sum()/base_varejo['Vendas'].sum()) * 100))

In [ ]:
#Vendas canceladas por País
df_vendas_pais = pd.DataFrame(base_varejo[base_varejo['Canceladas'] == 'Sim'].groupby('País')['Quantidade','Vendas'].sum()).reset_index().sort_values(by=['Vendas'])
df_vendas_pais["%"] = df_vendas_pais["Vendas"]/df_vendas_pais["Vendas"].sum() * 100
df_vendas_pais["%_Acc"] = df_vendas_pais["%"].cumsum()
df_vendas_pais = df_vendas_pais.reset_index(drop=True)
df_vendas_pais

In [ ]:
#cria um barplot, define os eixos e a paleta de cores usada para colorir as barras
plt.figure(figsize=(20,5))
ax = sns.barplot(x=df_vendas_pais["País"], y=abs(df_vendas_pais["Vendas"]), palette="RdYlBu")
ax.set_title("Análise de Pareto - Valor de vendas canceladas por País", fontsize = 20)
ax2 = ax.twinx()# Eixo secundário

# Cria a Curva de Pareto no eixo secundário
ax2.plot(df_vendas_pais["País"], df_vendas_pais["%_Acc"], color="g", marker="D", ms=7, label='Pareto')

# Configurações dos eixos 
ax2.tick_params(axis="y", colors="g")
ax2.set_ylim([0,110])

# Ajusta a inclinação dos labels do eixo x
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# Mostra os labels com a percentagem acumulada em cada ponto
gap = 5
for i in range(len(df_vendas_pais["%_Acc"])):
    ax2.annotate("%.1f" % df_vendas_pais["%_Acc"][i], (df_vendas_pais["País"][i], df_vendas_pais["%_Acc"][i] + gap))

In [ ]:
print('Valor de cancelamentos de venda no UK: ' + str(-base_varejo['Vendas'][(base_varejo['Canceladas'] == 'Sim') & (base_varejo['País'] == 'United Kingdom')].sum())+ str(' Libras'))

In [ ]:
#Qtd canceladas por País
df_qtd_pais = pd.DataFrame(base_varejo[base_varejo['Canceladas'] == 'Sim'].groupby('País')['Quantidade'].sum()).reset_index().sort_values(by=['Quantidade'])
df_qtd_pais["%"] = df_qtd_pais["Quantidade"]/df_qtd_pais["Quantidade"].sum() * 100
df_qtd_pais["%_Acc"] = df_qtd_pais["%"].cumsum()
df_qtd_pais = df_qtd_pais.reset_index(drop=True)
df_qtd_pais

In [ ]:
# Cria um barplot, define os eixos e a paleta de cores usada para colorir as barras
plt.figure(figsize=(20,5))
ax = sns.barplot(x=df_qtd_pais["País"], y=abs(df_qtd_pais["Quantidade"]), palette="RdYlBu")
ax.set_title("Análise de Pareto - Quantidade de itens cancelados por País", fontsize=20) # define e ajusta o tamanho da fonte do título

# Eixo secundário
ax2 = ax.twinx()

# Cria a Curva de Pareto no eixo secundário
ax2.plot(df_qtd_pais["País"], df_qtd_pais["%_Acc"], color="g", marker="D", ms=7, label='Pareto')

# Configurações dos eixos 
ax2.tick_params(axis="y", colors="g")
ax2.set_ylim([0,110])

# Ajusta a inclinação dos labels do eixo x
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

# Mostra os labels com a percentagem acumulada em cada ponto
gap = 5
for i in range(len(df_qtd_pais["%_Acc"])):
    ax2.annotate("%.1f" % df_qtd_pais["%_Acc"][i], (df_qtd_pais["País"][i], df_qtd_pais["%_Acc"][i] + gap))

In [ ]:
print('Qtd de cancelamentos de venda no UK: ' + str(-base_varejo['Quantidade'][(base_varejo['Canceladas'] == 'Sim') & (base_varejo['País'] == 'United Kingdom')].sum()))

### 4.0 United Kingdom - EDA

In [ ]:
base_varejo_uk = base_varejo[base_varejo['País'] == 'United Kingdom']
base_varejo_uk = base_varejo_uk.reset_index(drop=True)

In [ ]:
base_varejo_uk 

In [ ]:
# análise dos dados com profile
profile_base = ProfileReport(base_varejo_uk)
profile_base

In [ ]:
#métricas ipc 

#transações
total_transacoes_canceladas = base_varejo_uk['Fatura'][base_varejo_uk['Canceladas']=='Sim'].nunique()
total_transacoes = base_varejo_uk['Fatura'].nunique()
prop_transacoes_canceladas = (total_transacoes_canceladas/total_transacoes)*100

#itens
qtd_itens_cancelados = - base_varejo_uk['Quantidade'][base_varejo_uk['Canceladas']=='Sim'].sum()
qtd_itens_total = base_varejo_uk['Quantidade'].sum()
prop_qtd_cancelada = (qtd_itens_cancelados/qtd_itens_total)*100

#valor
valor_itens_cancelados = - base_varejo_uk['Vendas'][base_varejo_uk['Canceladas']=='Sim'].sum()
valor_itens_total = base_varejo_uk['Vendas'].sum()
prop_valor_cancelado = (valor_itens_cancelados/valor_itens_total)*100

# ticket médio (valor e qtd) e taket time (ritmo)
ticket_valor_transacao_cancelada = (valor_itens_cancelados)/total_transacoes_canceladas
ticket_qtd_transacao_cancelada = (qtd_itens_cancelados)/total_transacoes_canceladas
taket_time_valor_transacao_cancelada = (total_transacoes_canceladas/valor_itens_cancelados)
taket_time_qtd_transacao_cancelada = (total_transacoes_canceladas/qtd_itens_cancelados)

In [ ]:
#métricas ipc 

df_clientes = abs(base_varejo_uk[base_varejo_uk['Canceladas'] == 'Sim'].groupby(['ClienteID']).agg({'Vendas': 'sum','Fatura': 'nunique','Quantidade':'sum','Código_Estoque':'nunique'}))
df_clientes = df_clientes.rename(columns={'Vendas': 'Valor de cancelamentos','Fatura': 'Qtd de cancelamentos','Quantidade':'Qtd de itens cancelados', 'Código_Estoque':'Qtd_Itens_Dist'})
df_clientes = pd.DataFrame(df_clientes)
df_clientes

In [ ]:
print('Total de transações canceladas: ' + str(total_transacoes_canceladas))
print('\nProporção de transações canceladas: '+ str(prop_transacoes_canceladas))
print('\nQtd de itens cancelados: '+ str(qtd_itens_cancelados))
print('\nProporção de itens cancelados: '+ str(prop_qtd_cancelada))
print('\nValor de venda cancelado: '+ str(valor_itens_cancelados))
print('\nProporção do valor de venda de cancelado: '+ str(prop_valor_cancelado))
print('\nTicket médio em valor por transação cancelada: '+ str(ticket_valor_transacao_cancelada))
print('\nTicket médio em qtd por transação cancelada: '+ str(ticket_qtd_transacao_cancelada))
print('\nTaket time em valor por transação cancelada: '+ str(taket_time_valor_transacao_cancelada))
print('\nTaket time em qtd por transação cancelada: '+ str(taket_time_qtd_transacao_cancelada))
print('\nValor médio de cancelamentos para um cliente por transação: '+ str(df_clientes['Valor de cancelamentos'].mean()))
print('\nQuantidade média de cancelamentos para um cliente: '+ str(df_clientes['Qtd de cancelamentos'].mean()))
print('\nQuantidade média de itens cancelados para um cliente: '+ str(df_clientes['Qtd de itens cancelados'].mean()))
print('\nQuantidade média de itens distintos cancelados para um cliente: '+ str(df_clientes['Qtd_Itens_Dist'].mean()))

In [ ]:
#extrair dia da semana
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Data_Fatura'].dt.day_name()
base_varejo_uk['Dia_Semana'].unique()

In [ ]:
#transformar dia da semana
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Monday', 'Segunda')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Tuesday', 'Terça')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Wednesday', 'Quarta')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Thursday', 'Quinta')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Friday', 'Sexta')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Saturday', 'Sábado')
base_varejo_uk['Dia_Semana'] = base_varejo_uk['Dia_Semana'].replace('Sunday', 'Domingo')

In [ ]:
#Dia da semana com maior quantidade e valor de venda canceladas
#Sexta
pd.DataFrame(base_varejo_uk[base_varejo_uk['Canceladas'] == 'Sim'].groupby('Dia_Semana')['Quantidade','Vendas'].sum()).reset_index().sort_values(by=['Vendas'])

In [ ]:
#criar faixa horária
def extrair_faixa_hora(x):
    if x.hour >= 6 and x.hour < 12:
        return 'Manhã'
    elif x.hour >= 12 and x.hour < 18:
        return 'Tarde'
    elif x.hour >= 18 and x.hour < 24:
        return 'Noite'
    elif x.hour >= 0 and x.hour < 6:
        return 'Madrugada'

In [ ]:
base_varejo_uk['Faixa_Hora'] = [extrair_faixa_hora(base_varejo_uk['Hora_Fatura'][i]) for i in range(0, len(base_varejo_uk))]

In [ ]:
base_varejo_uk

In [ ]:
#Faixa horária com maior valor de venda e quantidade de itens de cancelados
#Manhã
pd.DataFrame(base_varejo_uk[base_varejo_uk['Canceladas'] == 'Sim'].groupby('Faixa_Hora')['Quantidade','Vendas'].sum()).reset_index().sort_values(by=['Vendas'])

### 5.0 Evolução dos cancelamentos

#### 5.1 Valor e quantidade de cancelamentos por ano

In [ ]:
df_uk = abs(base_varejo_uk[base_varejo_uk['Canceladas'] == 'Sim'].groupby(['Data_Fatura']).agg({'Fatura': 'nunique','Vendas': 'sum',}))
df_uk = abs(df_uk.rename(columns={'Fatura': 'Qtd de cancelamentos', 'Vendas': 'Valor de cancelamentos'}))
df_uk

In [ ]:
# define frequência mensal
df_uk_mes = df_uk[['Valor de cancelamentos','Qtd de cancelamentos']].resample('M').sum()
df_uk_mes

In [ ]:
#Evolução dos cancelamentos
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
plt.suptitle(" Valor e quantidade de cancelamentos por ano", fontsize=16)

df_uk_mes[['Valor de cancelamentos']]['2010'].plot(ax=axs[0,0], title='2010')
df_uk_mes[['Valor de cancelamentos']]['2011'].plot(ax=axs[0,1], title='2011')
df_uk_mes[['Qtd de cancelamentos']]['2010'].plot(ax=axs[1,0], title='2010')
df_uk_mes[['Qtd de cancelamentos']]['2011'].plot(ax=axs[1,1], title='2011')


plt.tight_layout()
plt.show()

#### 5.2 Ticket_Medio_Valor e Ticket_Medio_Qtd_Itens por ano

In [ ]:
df_uk2 = abs(base_varejo_uk[base_varejo_uk['Canceladas'] == 'Sim'].groupby(['Data_Fatura']).agg({'Vendas': 'sum','Fatura': 'nunique','Quantidade':'sum'}))
df_uk2 = df_uk2.rename(columns={'Vendas': 'Valor de cancelamentos','Fatura': 'Qtd de cancelamentos','Quantidade':'Qtd de itens cancelados'})
df_uk2

In [ ]:
df_uk2_mes = df_uk2[['Valor de cancelamentos','Qtd de cancelamentos','Qtd de itens cancelados']].resample('M').sum()
df_uk2_mes 

In [ ]:
df_uk2_mes['Ticket_Media_Valor'] = df_uk2_mes['Valor de cancelamentos']/df_uk2_mes['Qtd de cancelamentos']
df_uk2_mes['Ticket_Media_Qtd_Itens'] = df_uk2_mes['Qtd de itens cancelados']/df_uk2_mes['Qtd de cancelamentos']

In [ ]:
df_uk2_mes

In [ ]:
#Evolução dos cancelamentos
#Ticket_Medio_Valor e Ticket_Medio_Qtd_Itens por ano
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
plt.suptitle("Ticket_Medio_Valor e Ticket_Medio_QtdItens por ano", fontsize=16)

df_uk2_mes[['Ticket_Media_Valor']]['2010'].plot(ax=axs[0,0], title='2010')
df_uk2_mes[['Ticket_Media_Valor']]['2011'].plot(ax=axs[0,1], title='2011')
df_uk2_mes[['Ticket_Media_Qtd_Itens']]['2010'].plot(ax=axs[1,0], title='2010')
df_uk2_mes[['Ticket_Media_Qtd_Itens']]['2011'].plot(ax=axs[1,1], title='2011')


plt.tight_layout()
plt.show()

#### 5.3 Proporção de cancelamentos por ano

In [ ]:
df_uk3 = abs(base_varejo_uk.groupby(['Data_Fatura']).agg({'Canceladas': lambda x: (x == "Sim").nunique(),'Fatura': 'nunique'}))
df_uk3['Proporção'] = df_uk3['Canceladas']/df_uk3['Fatura']
df_uk3_mes = df_uk3[['Proporção']].resample('M').sum()
df_uk3_mes

In [ ]:
#Evolução de cancelamentos
#Proporção de cancelamentos por ano

plt.suptitle("Proporção de cancelamentos por ano", fontsize=16)

df_uk3_mes[['Proporção']]['2010'].plot(title='2010')
df_uk3_mes[['Proporção']]['2011'].plot(title='2011')

### 6.0 Produtos cancelados 

In [ ]:
df_produtos = base_varejo_uk[(base_varejo_uk['Canceladas'] == 'Sim')].groupby(['Código_Estoque','Descrição']).agg({'Vendas': 'sum','Fatura': 'nunique','Quantidade':'sum'}).sort_values(by=['Vendas'])
df_produtos["%"] = df_produtos['Vendas']/df_produtos['Vendas'].sum() * 100
df_produtos["%_Acc"] = df_produtos["%"].cumsum()

In [ ]:
df_produtos

In [ ]:
lista_codigos = ['M','AMAZONFEE','BANK CHARGES','POST','D','CRUK','S']

In [ ]:
df_produtos2 = base_varejo_uk[(base_varejo_uk['Canceladas'] == 'Sim') & (~base_varejo_uk['Código_Estoque'].isin(lista_codigos))].groupby(['Código_Estoque','Descrição']).agg({'Vendas': 'sum','Fatura': 'nunique','Quantidade':'sum'}).sort_values(by=['Vendas'])
df_produtos2["%"] = df_produtos2['Vendas']/df_produtos2['Vendas'].sum() * 100
df_produtos2["%_Acc"] = df_produtos2["%"].cumsum()

In [ ]:
df_produtos2

### 7.0 Modelo

#### 7.1 Preparação da base de dados

In [ ]:
base_varejo_uk

In [ ]:
base_modelo_aux = abs(base_varejo_uk.groupby(['Fatura','Dia_Semana','Faixa_Hora','Mês','ClienteID','Canceladas'])['Quantidade','Vendas','Preço','Código_Estoque'].agg({'Quantidade': 'sum','Vendas': 'sum','Preço':'mean','Código_Estoque':'nunique'}))
base_modelo_aux = base_modelo_aux.rename(columns={'Preço': 'Média_Preço','Código_Estoque':'Qtd_Itens_Distintos'})
base_modelo_aux = base_modelo_aux.reset_index()

In [ ]:
base_modelo_aux

In [ ]:
top50_percent_produtos = df_produtos2[df_produtos2['%_Acc']<= 50]
top50_percent_produtos = top50_percent_produtos.reset_index()
top50_percent_produtos

In [ ]:
lista_produtos = list(top50_percent_produtos['Código_Estoque'])
lista_produtos

In [ ]:
pivot_table = abs(pd.pivot_table(base_varejo_uk[base_varejo_uk['Código_Estoque'].isin(lista_produtos)], values='Quantidade', index=['Fatura','Dia_Semana','Faixa_Hora','Mês','ClienteID','Canceladas'], columns='Código_Estoque', aggfunc='sum'))
pivot_table = pivot_table.fillna(0)
pivot_table = pivot_table.reset_index()

In [ ]:
pivot_table 

In [ ]:
base_modelo = pd.merge(base_modelo_aux, pivot_table , on = ['Fatura','Dia_Semana','Faixa_Hora','Mês','ClienteID','Canceladas'], how = 'left')
base_modelo = base_modelo.fillna(0)

In [ ]:
#alterando tipo para int
base_modelo['20971'] = base_modelo['20971'].astype('int')
base_modelo['21108'] = base_modelo['21108'].astype('int')
base_modelo['22423'] = base_modelo['22423'].astype('int')
base_modelo['23113'] = base_modelo['23113'].astype('int')
base_modelo['23166'] = base_modelo['23166'].astype('int')
base_modelo['23843'] = base_modelo['23843'].astype('int')
base_modelo['47566B'] = base_modelo['47566B'].astype('int')
base_modelo['48185'] = base_modelo['48185'].astype('int')
base_modelo['71477'] = base_modelo['71477'].astype('int')
base_modelo['79323P'] = base_modelo['79323P'].astype('int')
base_modelo['79323W'] = base_modelo['79323W'].astype('int')
base_modelo['84078A'] = base_modelo['84078A'].astype('int')
base_modelo['85123A'] = base_modelo['85123A'].astype('int')

In [ ]:
base_modelo.info()

In [ ]:
base_modelo

#### 7.2 Treinamento do modelo

In [ ]:
# Split X e Y
X = base_modelo.drop(['Canceladas','Fatura'],axis = 1)
y = base_modelo.Canceladas

#split treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
cat_features = ['Dia_Semana', 'Faixa_Hora','ClienteID']

In [ ]:
# modelo
modelo = CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, cat_features=cat_features)

In [ ]:
# Defina a grade de hiperparâmetros a serem testados
params = {
    'iterations': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.1, 1],
    'depth': [4, 6, 8],
    }   

In [ ]:
#Criar objeto KFold com StratifiedKFold para validação cruzada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 1)

#Criar um objeto de grade de busca com GridSearchCV, a partir dos parâmetros que definimos
grid_search = GridSearchCV(modelo, param_grid = params, scoring = 'roc_auc',
                           cv = skf.split(X_train, y_train))

#Treinar o modelo com o grid search
grid_search.fit(X_train, y_train)

# Printar a melhor combinação de hiperparâmetros
print('Melhores hiperparâmetros:')
print(grid_search.best_params_)

In [ ]:
# Gera um modelo com as melhores combinações de hiperparâmetros
modelo = CatBoostClassifier(iterations=grid_search.best_params_['iterations'],
                        learning_rate=grid_search.best_params_['learning_rate'],
                        depth=grid_search.best_params_['depth'],
                        cat_features=cat_features)

#l2_leaf_reg=grid_search.best_params_['l2_leaf_reg'],
#border_count=grid_search.best_params_['border_count']

# Fita o modelo com o dataset de treino
modelo.fit(X_train, y_train)

In [ ]:
# previsão com os dados de teste
y_pred = modelo.predict(X_test)

# Calcular e printar métricas
# métricas
print("Acuracidade:\n")
print("Acurácia no conjunto de treino: {:.3f}".format(modelo.score(X_train, y_train)))
print("Acurácia no conjunto de teste: {:.3f}".format(modelo.score(X_test, y_test)))
print()

# O classification report traz diversas métricas interessantes, en formato de tabela, de forma simples
print("Métricas de classificação:\n")
print(classification_report(y_test, y_pred))
print()

# A matriz de confusão mostra de forma fácil os dados
# Verdadeiros Positivos | Falsos Positivos
# Falsos Negativos | Verdadeiros Negativos
print("Matriz de confusão:\n")
print(confusion_matrix(y_test, y_pred))

# Plotar importância das features
# Avalia quão importante é cada variável para as decisões que a árvore fornece
print()
print("Importância das variáveis:\n")
feature_scores = pd.Series(modelo.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_scores)

# plot
sns.barplot(x=feature_scores, y=feature_scores.index)

# labels
plt.xlabel('Importância das variáveis')
plt.ylabel('Variáveis')

# título
plt.title("Importância das variáveis")

# visualização
plt.show()

In [ ]:
# Converter os rótulos "Não" para 0 e "Sim" para 1
y_true_binary = [0 if label == "Não" else 1 for label in y_test]
y_pred_binary = [0 if label == "Não" else 1 for label in y_pred]

# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_binary)

# Calcular a área sob a curva ROC (ROC AUC)
roc_auc = roc_auc_score(y_true_binary, y_pred_binary)

In [ ]:
# Plotar a curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Taxa de falso positivo')
plt.ylabel('Taxa de verdadeiro positivo')
plt.title('Área sob a curva ROC')
plt.legend()
plt.show()

### 8.0 Modelo serializado

In [ ]:
# Salve o modelo em um arquivo pickle
with open('modelo.pkl', 'wb') as f:
    pickle.dump(modelo, f)